In [ ]:
# Load model directly
import transformers 
import torch
model_id = "meta-llama/Llama-3.1-8B-Instruct"
pipeline = transformers.pipeline("text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto",return_full_text=False)

In [3]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='en', target='ta')
translator.translate_batch(batch=["Hello, how are you?", "I am fine, thank you."], src="en", dest="ta")

['வணக்கம், நீங்கள் எப்படி இருக்கிறீர்கள்?', 'நான் நன்றாக இருக்கிறேன், நன்றி.']

In [ ]:
import asyncio
from googletrans import Translator
async def translate_bulk(inputs):
    translator = Translator()
    loop = asyncio.get_running_loop()

    # Run translate() in an executor to avoid blocking
    translations = await loop.run_in_executor(None, translator.translate, inputs, 'ta')

    # Ensure translations is always a list
    if not isinstance(translations, list):
        translations = [translations]

    for translation in translations:
        print(translation.origin, ' -> ', translation.text)

In [9]:
from datasets import load_dataset

from torch.utils.data import DataLoader
def collate_text(batch):
    return [item["text"] for item in batch]
ds = load_dataset("roneneldan/TinyStories")
dataloader = DataLoader(ds["train"], batch_size = 500, shuffle = False, collate_fn = collate_text)

In [10]:
batch =next(iter(dataloader))

In [11]:
from deep_translator import GoogleTranslator
translator = GoogleTranslator(source='en', target='ta')
translations = translator.translate_batch(batch)

In [8]:
translations[0]

'ஒரு நாள், லில்லி என்ற சிறுமி தனது அறையில் ஒரு ஊசியைக் கண்டாள். அது கூர்மையாக இருந்ததால் அதனுடன் விளையாடுவது கடினம் என்று அவளுக்குத் தெரியும். லில்லி தனது அம்மாவுடன் ஊசியைப் பகிர்ந்து கொள்ள விரும்பினார், அதனால் அவள் சட்டையில் ஒரு பொத்தானை தைக்க முடியும்.\n\nலில்லி தனது அம்மாவிடம் சென்று, "அம்மா, நான் இந்த ஊசியைக் கண்டேன். நீங்கள் அதை என்னுடன் பகிர்ந்து கொண்டு என் சட்டையை தைக்க முடியுமா?" அவளுடைய அம்மா புன்னகைத்து, "ஆம், லில்லி, நாங்கள் ஊசியைப் பகிர்ந்துகொண்டு உங்கள் சட்டையை சரிசெய்யலாம்" என்றார்.\n\nஒன்றாக, அவர்கள் ஊசியைப் பகிர்ந்துகொண்டு லில்லி சட்டையில் பொத்தானை தைத்தனர். அவர்கள் ஒருவருக்கொருவர் பகிர்ந்துகொண்டு உதவுவதால் அவர்களுக்கு இது கடினம் அல்ல. அவர்கள் முடிந்ததும், ஊசியைப் பகிர்ந்துகொண்டு தனது சட்டையை சரிசெய்ததற்காக லில்லி தனது அம்மாவுக்கு நன்றி தெரிவித்தார். அவர்கள் இருவரும் மகிழ்ச்சியாக உணர்ந்தார்கள், ஏனெனில் அவர்கள் பகிர்ந்து கொண்டனர் மற்றும் ஒன்றாக வேலை செய்தனர்.'

In [ ]:
inputs = [ds["train"][i]["text"] for i in range(1000)]
async def main():
    await translate_bulk(inputs)
asyncio.create_task(main())

In [ ]:
system_prompt = "You are a translator. I will tell you a story in words that a 4 year old can understand. You will translate it to Tamil for me"
message = ds["train"][-1]["text"]
final_input = [
    {"role": "assistant", "content": system_prompt},
    {"role": "user", "content": message}
]
print(final_input)

In [ ]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='gemma2', messages=final_input)
print(response['message']['content'])

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="abhinand/tamil-llama-7b-instruct-v0.2", device_map="auto")

In [ ]:

pipe(final_input)

In [ ]:
ஒருநாள், ஒரு சிறுமி சிலி (Lily) இன் அறையில் ஒரு துளையினை கண்டார். அவள் அது விளையாட முடியாது என்பதனாலும், அது கூரானதாக இருப்பதால் மட்டும் தெரிந்திருந்தது. சிலி அவளை தாய்விடம் பகிர்ந்து கொள்ள விரும்பினார், அ
அதனால் அவர் துணிகளில் ஒரு பட்டனை இணைக்கலாம். சிலி தாயாரிடம் சென்று "அம்மா, நான் ஒரு துளையினை கண்டேன். நீங்கள் எனது உடையில் ஒரு பட்டனை இணைக்க போதும்." என்று கூறினார். அம்மா சிரித்துப் பேசினார்: "ஆம் சிலி, ந
நாங்கள் துளையைப் பகிர்ந்து கொண்டு உன் உடையின் பட்டனை இணைக்கலாம்."

இருவரும் துளையினைப் பகிர்ந்து கொண்டு உடையில் பட்டனை இணைத்தார்கள். அது அவர்களுக்கு கடினமில்லை, ஏனென்றால் அவர்கள் ஒன்றுக்கொன்று உதவி செய்து கொண்டார்கள். முடிந்தவுடன் சிலி தாயாருக்கு துளையினைப் பகிர்ந்து கொ
கொடுத்தது மட்டுமல்லாமல், உடையின் பட்டனை இணைத்ததற்காகவும் நன்றி தெரிவித்தார். இருவரும் சேர்ந்து பணி செய்ததால் அந்த நிமிடம் மிகவும் மகிழ்ச்சியானதாக இருந்தது.


In [ ]:
from transformers import pipeline

pipe2 = pipeline("text-generation", model="roneneldan/TinyStories-8M")
print(pipe2("Once upon a time in a cave on top of a hill", max_new_tokens=400)[0]["generated_text"])

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe3 = pipeline("text-generation", model="roneneldan/TinyStories-33M")
print(pipe3("Once upon a time in a cave on top of a hill", max_new_tokens=400)[0]["generated_text"])

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="HuggingFaceTB/SmolLM2-135M", device_map="auto")
print(pipe("Once upon a time in a cave on top of a hill", max_new_tokens=400)[0]["generated_text"])
